In [ ]:
%load_ext autoreload
%autoreload 2
#%matplotlib qt

In [ ]:
import os
import sys
from pathlib import Path
sys.path.insert(1, str(Path(os.getcwd()).parent / "src"))

In [ ]:
from data_loader import DataLoader
from utils import *
from mcs_function import *
from plotting import *
import config
import pickle
import pmcx
import matplotlib.pyplot as plt
from mbll_functions import *
import pandas as pd
import scipy
import pickle

In [ ]:
pmcx.gpuinfo()

In [ ]:
config.gpuid

In [ ]:
config.simulated_dataset_dir

Determine tumor parameters via linear least squares from attenuation data.

In [ ]:
#tumor_df = pd.read_excel(
#    config.spectra_dir / "tumor_optical_properties.xlsx",
#    sheet_name="HGG_fresh", # high-grade glioma, fresh
#)
tumor_df = pd.read_excel(
    config.spectra_dir / "tumor_optical_properties.xlsx",
    sheet_name="HGG_comparison"
)
tumor_df_wavelength_condition = (tumor_df["Wavelength"] >= 400) & (tumor_df["Wavelength"] <= 1000) 
tumor_absorption_spectrum = tumor_df.loc[tumor_df_wavelength_condition, "mua_Gebhart_frozen_freshly_excised_snapfrozen"].to_numpy() * 10
tumor_scattering_spectrum = tumor_df.loc[tumor_df_wavelength_condition, "mus_fitted_present_fresh_30mins"].to_numpy() * 10
wavelengths_tumor = tumor_df.loc[tumor_df_wavelength_condition, "Wavelength"].to_numpy()

In [ ]:
mu_a_tumor = DataLoader.absorption_coefs(
    wavelengths_tumor,
    use_diff_oxycco=False,
    use_water_and_fat=True
)

In [ ]:
res = scipy.optimize.lsq_linear(
    mu_a_tumor,
    tumor_absorption_spectrum,
    bounds=(
        [0, 0, 0, 0, 0, 0],
        [1, 1, 0.1, 0.1, 1, 1]
    )
)
tumor_base_concentrations = res.x
print(tumor_base_concentrations)
print(DataLoader.tissue_parameters["gray matter"][0])

The oxCCO value seems much to high. There might be another chromophore that I am not accounting for. I will instead simply use tissue concentrations as for white matter. White matter tissue was reported to have very similar absorption spectra to brain tumor.

In [ ]:
# simulated tissue concentrations
simulation_data = dict()
for tissue_type in DataLoader.tissue_parameters.keys():
    base_concentrations_blood_fraction = concentrations_to_blood_fraction(DataLoader.tissue_parameters[tissue_type][0])
    base_scattering_params = DataLoader.tissue_parameters[tissue_type][1]
    sampled_concentrations = np.random.multivariate_normal(
        mean=base_concentrations_blood_fraction,
        cov=np.diag([0.0012, 0.0012, 1e-6, 1e-6, 0.0012, 0.0012]),
        size=(10,)
    )
    sampled_scattering_params = np.random.multivariate_normal(
        mean=base_scattering_params,
        cov=np.diag([25, 0.0225]),
        size=(10,)
    )
    # clip values
    sampled_concentrations = np.clip(sampled_concentrations, a_min=[[0, 0, 0, 0, 0, 0]], a_max=[[1, 1, np.inf, np.inf, 1, 1]])
    sampled_scattering_params = np.clip(sampled_scattering_params, a_min=[[0, 0]], a_max=[[200, 10]])
    # prepend baseline concentrations to data
    sampled_concentrations = np.row_stack((base_concentrations_blood_fraction, sampled_concentrations))
    sampled_scattering_params = np.row_stack((base_scattering_params, sampled_scattering_params))
    simulation_data[tissue_type] = (sampled_concentrations, sampled_scattering_params)

MCX Configuration

In [ ]:
# empty log file
open(config.simulated_dataset_dir / "log.txt", "w").close()

# make sure to make this a separate array, don't create directly in cfg!
prop = np.array([[0,0,1,1],[0,0,5,5]])

vol = np.ones((100, 100, 120), dtype="uint8")

cfg = {
    "nphoton": 1e7,
    "maxdetphoton": 1e7,
    "vol": vol,
    "tstart":0,
    "tend":5e-7,
    "tstep":5e-7,
    "autopilot": 1,
    "gpuid": config.gpuid,
    "prop": prop,
    "bc": "ccrcca",#001000",
    "isspecular": 1,
    "srcdir": [0,0,1],
    "srctype": "planar",
    "srcpos": [0, 0, 0],
    "srcparam1": [vol.shape[0], 0, 0, 0], # 3D position of vertex, fourth coordinate is irrelevant
    "srcparam2": [0, vol.shape[1], 0, 0],
    "issrcfrom0": 0,
    #"savedetflag": "dpx",
    "flog": str(config.simulated_dataset_dir) + "/log.txt",
    "isnormalized": 0,
    "issaveref": 1
}

# necessary to accumulate weights
cfg["vol"][:, :, 0] = 0

In [ ]:
wavelengths = np.arange(520, 905, 100)
mu_a_matrix = DataLoader.absorption_coefs(
    wavelengths,
    use_diff_oxycco=False,
    use_water_and_fat=True
)

In [ ]:
for tissue_type in simulation_data.keys():
    tissue_spectra = []
    g, refractive_idx = DataLoader.tissue_parameters[tissue_type][-2:]
    for concentrations_blood_fraction, scattering_params in zip(*simulation_data[tissue_type][:2]):
        mu_a_vals = mu_a_matrix @ blood_fraction_to_concentrations(concentrations_blood_fraction)
        mu_s_red_vals = scattering_params[0] * (wavelengths / 500) ** (-scattering_params[1])
        mu_a_vals /= 10
        mu_s_red_vals /= 10
        spectrum = []
        for mu_a, mu_s_red in zip(mu_a_vals, mu_s_red_vals):
            cfg["prop"][1, 0] = mu_a
            cfg["prop"][1, 1] = mu_s_red / (1-g)
            cfg["prop"][1, 2] = g
            cfg["prop"][1, 3] = refractive_idx
            res = pmcx.mcxlab(cfg)
            dref = np.sum(res["dref"][:, :, 0, :]) / cfg["nphoton"]
            attenuation = -np.log(dref)
            spectrum.append(attenuation)
        tissue_spectra.append(spectrum)
    simulation_data[tissue_type] = simulation_data[tissue_type][:2] + (np.array(tissue_spectra),)

Save data to file...

In [ ]:
with open(config.simulated_dataset_dir / "simulation_data.pickle", "wb") as f:
    pickle.dump(simulation_data, f)

In [ ]:
plot_spectra_slider(simulation_data["gray_matter"][-1], wavelengths)